In [2]:
!pip install transformers
!pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [3]:
text = open("MonekyPawsStory - Copy.txt", "r")

In [4]:
read = text.read()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 457: character maps to <undefined>

In [ ]:
import textwrap

wrapper = textwrap.TextWrapper(width=80, break_long_words=False, break_on_hyphens=False)
print(wrapper.fill(read))
modifiedText = wrapper.fill(read)

In [ ]:
from transformers import pipeline


model = "google-t5/t5-small"
pipe = pipeline("summarization", model=model, tokenizer=model, device=-1)
outputs = pipe(modifiedText, max_length=512, clean_up_tokenization_spaces=True)

In [ ]:
print(wrapper.fill(outputs[0]['summary_text']))

In [ ]:
summary1 = open("Summary1.txt", "r") #AI Summary
summary2 = open("Summary2.txt", "r") #Wikipedia Summary
summary3 = open("Summary3.txt", "r") #My Summary

In [ ]:
summary1_read = summary1.read()
summary2_read = summary2.read()
summary3_read = summary3.read()

In [ ]:
print(wrapper.fill(summary1_read))
summary1Mod = wrapper.fill(summary1_read)

In [ ]:
print(wrapper.fill(summary2_read))
summary2Mod = wrapper.fill(summary2_read)

In [ ]:
print(wrapper.fill(summary3_read))
summary3Mod = wrapper.fill(summary3_read)

In [ ]:
from rouge_score import rouge_scorer

generated_summary = wrapper.fill(outputs[0]['summary_text'])
reference_summaries = [summary1Mod, summary2Mod, summary3Mod]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

scores = {key: [] for key in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']}
for ref in reference_summaries:
    temp_scores = scorer.score(ref, generated_summary)
    for key in temp_scores:
        scores[key].append(temp_scores[key])

for key in scores:
    print(f'{key}:\n{scores[key]}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from rouge_score import rouge_scorer

all_scores = []
for ref in reference_summaries:
    temp_scores = scorer.score(ref, generated_summary)
    all_scores.append(temp_scores)

rouge_types = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
metrics = ['precision', 'recall', 'fmeasure']
num_references = len(all_scores)

plot_data = {rt: {m: [] for m in metrics} for rt in rouge_types}

for scores_for_ref in all_scores:
    for rt in rouge_types:
        plot_data[rt]['precision'].append(scores_for_ref[rt].precision)
        plot_data[rt]['recall'].append(scores_for_ref[rt].recall)
        plot_data[rt]['fmeasure'].append(scores_for_ref[rt].fmeasure)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))
axes = axes.flatten()

bar_width = 0.2
index = np.arange(num_references)

for i, rt in enumerate(rouge_types):
    ax = axes[i]
    ax.bar(index, plot_data[rt]['precision'], bar_width, label='Precision')
    ax.bar(index + bar_width, plot_data[rt]['recall'], bar_width, label='Recall')
    ax.bar(index + 2 * bar_width, plot_data[rt]['fmeasure'], bar_width, label='F-measure')

    ax.set_xlabel('Reference Summary')
    ax.set_ylabel('Score')
    ax.set_title(f'ROUGE {rt.upper()} Scores')
    ax.set_xticks(index + bar_width)
    ax.set_xticklabels([f'summary{j+1}' for j in range(num_references)])
    ax.legend()
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()